# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [1]:
# Importo las librerías que voy a necesitar
import requests  # Para hacer las peticiones a la página web
from bs4 import BeautifulSoup  # Para "entender" y poder buscar cosas en el HTML
import pandas as pd  # Para crear la tabla (DataFrame) al final
import time # Para añadir pausas y no saturar el servidor

# Definición de la función principal

def scrape_books(min_rating, max_price):
    """
    Esta función hace web scraping en la web 'books.toscrape.com'.
    Busca libros que cumplan con una valoración mínima y un precio máximo,
    y devuelve los datos en un DataFrame de pandas.
    """

    # Diccionario para convertir la clase de la estrella (texto) a un número.
    # Por ejemplo, la clase 'star-rating Four' la convertiré al número 4.
    rating_map = {
        'One': 1,
        'Two': 2,
        'Three': 3,
        'Four': 4,
        'Five': 5
    }

    # Lista donde iré guardando los datos de cada libro que cumpla las condiciones.
    lista_libros_ok = []

    # Defino la URL base y la primera página por la que empezar.
    url_base = 'http://books.toscrape.com/'
    # La página 1 no tiene "page-1.html", pero las siguientes sí, así que empiezo por la 2 y
    # trato la 1 como un caso especial. O mejor, empiezo directamente por la primera página del catálogo.
    url_pagina_actual = url_base + 'catalogue/page-1.html'
    
    print(f"Iniciando el scraping...")
    print(f"Condiciones: Valoración >= {min_rating} y Precio <= £{max_price}")

    # Bucle para recorrer todas las páginas del catálogo.
    # Usaré un bucle 'while True' que se detendrá cuando no encuentre el botón "siguiente".
    while True:
        print(f"Analizando página: {url_pagina_actual}")
        
        try:
            # Hago la petición a la página actual.
            respuesta = requests.get(url_pagina_actual)
            # Si la página no responde bien, salto a la siguiente iteración.
            if respuesta.status_code != 200:
                print(f"Error al acceder a la página {url_pagina_actual}. Saltando...")
                break
        except requests.exceptions.RequestException as e:
            print(f"Error de conexión: {e}")
            break

        # Creo el objeto BeautifulSoup para poder analizar el HTML.
        sopa_pagina = BeautifulSoup(respuesta.content, 'html.parser')

        # Encuentro todos los libros en la página. Cada libro está en una etiqueta <article>.
        libros_en_pagina = sopa_pagina.find_all('article', class_='product_pod')

        # Ahora, para cada libro de la página, tengo que entrar en su enlace para ver los detalles.
        for libro_articulo in libros_en_pagina:
            
            # Obtengo el enlace a la página del libro. El enlace es relativo, así que tengo que unirlo a la URL base.
            enlace_libro_relativo = libro_articulo.find('h3').find('a')['href']
            # Construyo la URL completa para la página del detalle del libro.
            url_libro_completa = url_base + 'catalogue/' + enlace_libro_relativo.replace('../', '')

            # Hago una pausa pequeña para no sobrecargar el servidor de la página.
            time.sleep(0.1) 
            
            # Ahora hago una nueva petición para obtener la página de detalle del libro.
            try:
                respuesta_libro = requests.get(url_libro_completa)
                if respuesta_libro.status_code != 200:
                    continue # Si falla, simplemente sigo con el siguiente libro.
            except requests.exceptions.RequestException:
                continue
                
            sopa_libro = BeautifulSoup(respuesta_libro.content, 'html.parser')

            # Extraigo el precio y la valoración para ver si cumple mis condiciones.
            precio_texto = sopa_libro.find('p', class_='price_color').text
            precio_float = float(precio_texto.replace('£', '')) # Lo convierto a número

            clase_rating = sopa_libro.find('p', class_='star-rating')['class'][1] # La clase es "star-rating Five", me quedo con "Five"
            rating_num = rating_map.get(clase_rating, 0) # Lo convierto a número (si no lo encuentra, pone 0)

            # FILTRO
            # Compruebo si el libro cumple las condiciones de rating y precio.
            if rating_num >= min_rating and precio_float <= max_price:
                # Si las cumple, extraigo el resto de la información.
                
                titulo = sopa_libro.find('h1').text
                
                # Para UPC, Genre, y Availability, están en una tabla. Tengo que buscar en ella.
                tabla = sopa_libro.find('table', class_='table table-striped')
                filas_tabla = tabla.find_all('tr')
                
                upc = filas_tabla[0].find('td').text
                genero = filas_tabla[1].find('td').text
                disponibilidad = filas_tabla[5].find('td').text
                
                # La descripción está justo después del div con el id 'product_description'
                try:
                    descripcion = sopa_libro.find('div', id='product_description').find_next_sibling('p').text
                except AttributeError:
                    descripcion = "No disponible" # Por si algún libro no tiene descripción

                # Guardo toda la info en un diccionario
                info_libro = {
                    'UPC': upc,
                    'Title': titulo,
                    'Price (£)': precio_float,
                    'Rating': rating_num,
                    'Genre': genero,
                    'Availability': disponibilidad.strip(),
                    'Description': descripcion
                }
                
                # Añado el diccionario a mi lista de libros que cumplen las condiciones.
                lista_libros_ok.append(info_libro)
                print(f"  -> ¡Encontrado! '{titulo}' cumple las condiciones.")


        # Al final del bucle de los libros, busco el botón "siguiente".
        siguiente_boton = sopa_pagina.find('li', class_='next')
        if siguiente_boton:
            # Si existe, construyo la URL de la siguiente página y el bucle 'while' continuará.
            enlace_siguiente = siguiente_boton.find('a')['href']
            url_pagina_actual = url_base + 'catalogue/' + enlace_siguiente
        else:
            # Si no hay botón "siguiente", significa que hemos llegado al final. Rompo el bucle.
            print("No hay más páginas. Terminando el bucle.")
            break
            
    print("Scraping completado. Creando el DataFrame...")

    # Con la lista de diccionarios, creo el DataFrame de pandas.
    df_resultado = pd.DataFrame(lista_libros_ok)
    
    return df_resultado

# Ejecución del script

# Esto es una buena práctica para que el código solo se ejecute cuando corro este archivo directamente.
if __name__ == "__main__":
    
    # Defino las condiciones que me pide el lab.
    MIN_RATING = 4
    MAX_PRICE = 20

    # Llamo a mi función para que haga todo el trabajo.
    df_libros = scrape_books(min_rating=MIN_RATING, max_price=MAX_PRICE)

    # Muestro el resultado final.
    print("Libros encontrados que cumplen las condiciones")
    print(df_libros)

    # Opcional: guardo el resultado en un archivo CSV.
    if not df_libros.empty:
        df_libros.to_csv('libros_filtrados.csv', index=False)
        print("\nEl resultado se ha guardado en 'libros_filtrados.csv'")

Iniciando el scraping...
Condiciones: Valoración >= 4 y Precio <= £20
Analizando página: http://books.toscrape.com/catalogue/page-1.html
  -> ¡Encontrado! 'Set Me Free' cumple las condiciones.
Analizando página: http://books.toscrape.com/catalogue/page-2.html
  -> ¡Encontrado! 'The Four Agreements: A Practical Guide to Personal Freedom' cumple las condiciones.
  -> ¡Encontrado! 'Sophie's World' cumple las condiciones.
Analizando página: http://books.toscrape.com/catalogue/page-3.html
  -> ¡Encontrado! 'Untitled Collection: Sabbath Poems 2014' cumple las condiciones.
  -> ¡Encontrado! 'This One Summer' cumple las condiciones.
  -> ¡Encontrado! 'Thirst' cumple las condiciones.
Analizando página: http://books.toscrape.com/catalogue/page-4.html
Analizando página: http://books.toscrape.com/catalogue/page-5.html
  -> ¡Encontrado! 'Princess Jellyfish 2-in-1 Omnibus, Vol. 01 (Princess Jellyfish 2-in-1 Omnibus #1)' cumple las condiciones.
  -> ¡Encontrado! 'Princess Between Worlds (Wide-Awake P